<a href="https://colab.research.google.com/github/YellaCRE/FIFA_RecommandSystem/blob/main/FIFA_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 피파온라인4 선수추천알고리즘_전처리 파이프라인

In [ ]:
import pandas as pd
from pandas import DataFrame
import re
from tqdm import tqdm
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#player_df=pd.read_csv('D:/player_df.csv')
player_df = pd.read_csv("/content/drive/MyDrive/2022.2 피파선수 추천 프로젝트/player_df.csv")

#기본 전처리

포지션/스탯 분리

In [ ]:
player_df.insert(5,"포지션",0)
player_df.insert(6,"스탯",0)
player_df.iat[0,6]=re.findall("\d+", player_df['포지션/스탯'][0])[0]

for i in tqdm(range(len(player_df))):
    player_df.iat[i,5]=re.findall("[a-zA-Z]+", player_df['포지션/스탯'][i])[0]
    player_df.iat[i,6]=re.findall("\d+", player_df['포지션/스탯'][i])[0]

player_df=player_df.drop(['포지션/스탯'],axis=1)
col_name = ['index','id','name','시즌','포지션','스탯', '키', '몸무게', '속력', '가속력', '골 결정력', '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '페널티 킥', '짧은 패스', '시야', '크로스', '긴 패스', '프리킥', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도', '대인 수비', '태클', '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프', '침착성', 'GK 다이빙', 'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정','개인기', '특성', '급여', '주발/약발', '체형', '국적', '클럽경력','출전','득점','도움','유효슈팅','일반슈팅','패스성공률','드리블성공률','공중볼경합성공률','가로채기','태클성공률','차단성공률','선방','평점']
player_df=player_df[col_name]

100%|██████████| 1000/1000 [00:00<00:00, 2704.05it/s]


주발/약발 나누기

In [ ]:
player_df.insert(46,"left",0)
player_df.insert(47,"right",0)
player_df=player_df.loc[:,~player_df.T.duplicated()]
player_df.insert(46,"right",0)

for i in tqdm(range(len(player_df))):
    player_df.iat[i,46]=re.findall("\d", player_df['주발/약발'][i])[0]
    player_df.iat[i,47]=re.findall("\d", player_df['주발/약발'][i])[1]

player_df=player_df.drop(['주발/약발'],axis=1)

100%|██████████| 1000/1000 [00:00<00:00, 3510.99it/s]


특성 onehotencoding

In [ ]:
#결측값 처리
player_df = player_df.fillna('없음')

player_hidden_iter = (set(x.split(',')) for x in player_df.특성)
player_hidden_set = sorted(set.union(*player_hidden_iter))
print(player_hidden_set)
indicator_mat = DataFrame(np.zeros((len(player_df), len(player_hidden_set))), columns=player_hidden_set)

for i, j in enumerate(player_df.특성):
    indicator_mat.loc[i, j.split(',')] = 1

player_df=player_df.drop(['특성'],axis=1)

['GK공격가담', 'GK능숙한펀칭', 'GK멀리던지기', 'GK소극적크로스수비', 'GK적극적크로스수비', 'GK침착한1:1수비', '강철몸', '개인플레이선호', '긴패스선호', '다이버', '스위퍼키퍼', '스피드드리블러', '슬라이딩태클선호', '승부욕', '아웃사이드슈팅/크로스', '얼리크로스선호', '없음', '예리한감아차기', '유리몸', '장거리스로잉', '주발선호', '중거리슛선호', '초장거리스로인', '칩슛선호', '테크니컬드리블러', '트러블메이커', '파워헤더', '패스마스터', '플레이메이커', '화려한개인기', '화려한걷어내기']


체형

In [ ]:
player_df['체형']=player_df['체형'].replace('마름','90')
player_df['체형']=player_df['체형'].replace('보통','100')
player_df['체형']=player_df['체형'].replace('건장','110')

키 몸무게 단위 제거

In [ ]:
player_df['키'] = player_df['키'].str.replace('cm', '').astype('int64')
player_df['몸무게'] = player_df['몸무게'].str.replace('kg', '').astype('int64')

#강화 적용

In [ ]:
num_coloums = ['index', 'id', 'name', '시즌', '포지션', '스탯', '키', '몸무게', '속력', '가속력',
              '골 결정력', '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '페널티 킥', '짧은 패스', '시야',
              '크로스', '긴 패스', '프리킥', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도',
              '대인 수비', '태클', '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프',
              '침착성', 'GK 다이빙', 'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정', '개인기', '급여',
              'right', 'left', '체형']
stat_coloums = ['스탯','속력', '가속력', '골 결정력', '슛 파워', '중거리 슛', '위치 선정', '발리 슛', 
                '페널티 킥','짧은 패스', '시야', '크로스', '긴 패스', '프리킥', '커브', '드리블', '볼 컨트롤', 
                '민첩성', '밸런스','반응 속도', '대인 수비', '태클', '가로채기', '헤더', '슬라이딩 태클', '몸싸움', 
                '스태미너', '적극성','점프', '침착성', 'GK 다이빙', 'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정']
                
player_num = pd.concat([player_df[num_coloums],indicator_mat], axis=1)
player_stats=player_num[stat_coloums].astype('int64')

강화 스탯 적용

In [ ]:
#순서대로 1강
player_stats1 = player_stats+3 #0강에서 1강으로 갈 때 전체 스탯 상승폭은 3이다
player_stats2 = player_stats1+1
player_stats3 = player_stats2+1
player_stats4 = player_stats3+2
player_stats5 = player_stats4+2
player_stats6 = player_stats5+2
player_stats7 = player_stats6+3
player_stats8 = player_stats7+4
player_stats9 = player_stats8+4
player_stats10 = player_stats9+5

In [ ]:
#강화 수치 리스트 생성
for n in range(1,11): 
  globals()['강화'+str(n)] = []
  for i in range(len(player_num)):
    globals()['강화'+str(n)].append(n)

#순서 재배열
indicator_name = player_num.columns[47:]
ID=player_num[['index', 'id', 'name', '시즌', '포지션', '키', '몸무게']]
indicator=player_num[indicator_name]
filter=player_num[['개인기', 'left', 'right','급여','체형']]

#강화 수치별 데이터프레임 형성
for n in range(1,11): 
  globals()['player_num'+str(n)] = pd.concat([ID,globals()['player_stats'+str(n)],filter,pd.DataFrame(globals()['강화'+str(n)]),indicator],axis=1)

#최종 데이터 형식 player_num_total
player_num_total = pd.DataFrame()
for n in range(1,11):
  player_num_total = pd.concat([player_num_total,globals()['player_num'+str(n)]])

player_num_total.rename(columns={0:'강화'},inplace=True)

# csv 변환

In [ ]:
player_num_total.to_csv("player_num_total.csv", mode='w', index=False, encoding="utf-8-sig")